In [ ]:
# when using google drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
dataset_path = "/content/gdrive/MyDrive/Kelompok E Anamedsos/CODE/1 - TWEET COLLECTION/"
path = "/content/gdrive/MyDrive/Kelompok E Anamedsos/CODE/1 - STANCE DETECTION/"

In [ ]:
!pip install nltk
!pip install plotly
!pip install gensim
!pip install tqdm
!pip install xgboost

import numpy as np
import nltk
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import OrderedDict, Counter
from wordcloud import WordCloud
from tqdm import tqdm

from gensim import corpora, models, similarities
from gensim.models import CoherenceModel

from imblearn.over_sampling import SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")

sns.set_style("darkgrid")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Data Loading

In [ ]:
df = pd.read_csv(dataset_path+"twint-dataset/vaksin-twint-filter.csv")
data_label_df = pd.read_csv(dataset_path+"twint-dataset/vaksin-twint-label-complete.csv")

In [ ]:
df.sort_values('id', ascending=True)

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
10700,1526873255583154177,1526852598069833730,1.652870e+12,2022-05-18 10:32:06,0,NaN,@ftrrii1 @husnaa_a Pandemi berdasarkan dugaan ...,in,[],[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'ftrrii1', 'name': 'sweet chi...",NaN,NaN,NaN,NaN
10699,1526873332393598977,1526873332393598977,1.652870e+12,2022-05-18 10:32:25,0,NaN,Penumpang KAJJ vaksinasi dosis kedua atau boos...,in,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
10698,1526873365033537536,1526816622245052417,1.652870e+12,2022-05-18 10:32:32,0,NaN,@its_shihaa Udaah pake pfizer jumat minggu lal...,in,[],[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'its_shihaa', 'name': '𝐒𝐡𝐢𝐡𝐚 ...",NaN,NaN,NaN,NaN
10697,1526874139381755904,1526874139381755904,1.652870e+12,2022-05-18 10:35:37,0,NaN,VIDEO: FDA Izinkan Vaksin Booster Pfizer untuk...,in,"['terbaru', 'anak', 'booster', 'fda']",[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
10696,1526875038070218752,1526875038070218752,1.652870e+12,2022-05-18 10:39:11,0,NaN,"Percepat Vaksinasi, Dengan Humanis Forpimka Se...",in,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,1528160460423708672,1528160460423708672,1.653177e+12,2022-05-21 23:47:00,0,NaN,"""Garis panduan Pertubuhan Kesihatan Sedunia (W...",in,"['icymi', 'awaninews', 'awanipagi']",[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1528160751445086209,1528160751445086209,1.653177e+12,2022-05-21 23:48:09,0,NaN,Step by step.... Outputnya ya vaksin. Simpel. ...,in,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1528161749060440066,1528161749060440066,1.653177e+12,2022-05-21 23:52:07,0,NaN,Njir udah gejala meriang gegara efek vaksin bo...,in,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1528161907710251008,1528161907710251008,1.653177e+12,2022-05-21 23:52:45,0,NaN,Mager nya efek vaksin ini 🥲🥲🥲,in,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [ ]:
df.sort_values('date')

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
10700,1526873255583154177,1526852598069833730,1.652870e+12,2022-05-18 10:32:06,0,NaN,@ftrrii1 @husnaa_a Pandemi berdasarkan dugaan ...,in,[],[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'ftrrii1', 'name': 'sweet chi...",NaN,NaN,NaN,NaN
10699,1526873332393598977,1526873332393598977,1.652870e+12,2022-05-18 10:32:25,0,NaN,Penumpang KAJJ vaksinasi dosis kedua atau boos...,in,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
10698,1526873365033537536,1526816622245052417,1.652870e+12,2022-05-18 10:32:32,0,NaN,@its_shihaa Udaah pake pfizer jumat minggu lal...,in,[],[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'its_shihaa', 'name': '𝐒𝐡𝐢𝐡𝐚 ...",NaN,NaN,NaN,NaN
10697,1526874139381755904,1526874139381755904,1.652870e+12,2022-05-18 10:35:37,0,NaN,VIDEO: FDA Izinkan Vaksin Booster Pfizer untuk...,in,"['terbaru', 'anak', 'booster', 'fda']",[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
10696,1526875038070218752,1526875038070218752,1.652870e+12,2022-05-18 10:39:11,0,NaN,"Percepat Vaksinasi, Dengan Humanis Forpimka Se...",in,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,1528160460423708672,1528160460423708672,1.653177e+12,2022-05-21 23:47:00,0,NaN,"""Garis panduan Pertubuhan Kesihatan Sedunia (W...",in,"['icymi', 'awaninews', 'awanipagi']",[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1528160751445086209,1528160751445086209,1.653177e+12,2022-05-21 23:48:09,0,NaN,Step by step.... Outputnya ya vaksin. Simpel. ...,in,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1528161749060440066,1528161749060440066,1.653177e+12,2022-05-21 23:52:07,0,NaN,Njir udah gejala meriang gegara efek vaksin bo...,in,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1528161907710251008,1528161907710251008,1.653177e+12,2022-05-21 23:52:45,0,NaN,Mager nya efek vaksin ini 🥲🥲🥲,in,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [ ]:
drop_idx = data_label_df[data_label_df['stance']=='SKIP'].index
data_label_df.drop(drop_idx, inplace=True)

keywords = ['hepatitis', 'kucing', 'cacar', 'kitten', 'serviks', 'anabul']
drop_idx = []

for idx, row in df.iterrows():
  tweet = row['tweet'].lower()
  is_invalid = False

  for keyword in keywords:
    if keyword in tweet:
      is_invalid = True
      break
  
  if is_invalid:
    drop_idx.append(idx)

df.drop(drop_idx, inplace=True)

In [ ]:
complete_df = df.merge(data_label_df[['id', 'stance']], 'left', right_on='id', left_on='id')
complete_df['stance'].fillna(value='UNLABELED', inplace=True)

In [ ]:
complete_df.sample(5)

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,stance
3159,1527602672903200773,1527602672903200773,1.653044e+12,2022-05-20 10:50:33,0,NaN,Pelanggan KA Jarak Jauh yang telah mendapatkan...,in,[],[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,UNLABELED
1684,1527848466541514754,1527848466541514754,1.653102e+12,2022-05-21 03:07:15,0,NaN,"Percayakan vaksinasi kepada pemerintah, jangan...",in,[],[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,UNLABELED
1300,1527876210231214081,1527856710312132608,1.653109e+12,2022-05-21 04:57:29,0,NaN,@jembermfs Tujuan Jimbaran Bbm 60k pelabuhan 3...,in,[],[],...,NaN,NaN,NaN,NaN,"[{'screen_name': 'jembermfs', 'name': 'BASE JE...",NaN,NaN,NaN,NaN,UNLABELED
23,1528153979988762625,1527641427068735489,1.653175e+12,2022-05-21 23:21:15,0,NaN,@KemenkesRI Data ini darimana @KemenkesRI ?? J...,in,[],[],...,NaN,NaN,NaN,NaN,"[{'screen_name': 'KemenkesRI', 'name': 'Kement...",NaN,NaN,NaN,NaN,UNLABELED
9092,1526950878611382272,1526950878611382272,1.652888e+12,2022-05-18 15:40:33,0,NaN,habis vaksin otak merosot ke dengkul😤,in,[],[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,UNLABELED


# Data Preprocessing

Memilih kolom data yang akan dipakai

In [ ]:
selected_df = complete_df[['id', 'tweet', 'stance']]

Proses pembersihan data

In [ ]:
def clean_data(tweet):
    # lowercase
    normal_tw = tweet.lower()
    # hapus b'
    normal_tw = re.sub(r'^b\'', '', normal_tw)
    # hapus RT
    normal_tw = re.sub(r'^rt ', '', normal_tw)
    # hapus emoji
    normal_tw = re.sub(r'\\x.{2}', '', normal_tw)
    # hapus www.* atau https?://* (URL)
    normal_tw = re.sub(r'((www\.[^\s]*)|(https?://[^\s]*))', '', normal_tw)
    # remove spasi berlebih
    normal_tw = re.sub(r'\s+', ' ', normal_tw)
    # trim depan belakang
    normal_tw = normal_tw.strip()
    # regex huruf yang berulang kaya haiiii (untuk fitur unigram)
    normal_regex = re.compile(r"(.)\1{1,}")
    # buang huruf yang berulang
    normal_tw = normal_regex.sub(r"\1\1", normal_tw)
    # hapus @username
    normal_tw = re.sub(r'@[^\s]+', '', normal_tw)
    # hapus hashtag
    normal_tw = re.sub(r'#[^\s]+', '', normal_tw)
    # hapus tanda baca
    normal_tw = re.sub(r'[^\w\s]', '', normal_tw) 
    # hapus angka
    normal_tw = re.sub(r'\d+', ' ', normal_tw) 
    return normal_tw

def remove_stopwords_and_normalize(tweet):
    token = nltk.word_tokenize(tweet)
    token_new = []
    for k in token:
        if k in df_kamus_singkatan['singkatan'].values:
            k = df_kamus_singkatan.loc[df_kamus_singkatan['singkatan']
                                       == k, 'asli'].values[0]
        if k in df_kamus_alay['slang'].values:
            k = df_kamus_alay.loc[df_kamus_alay['slang']
                                  == k, 'formal'].values[0]
        if k not in stopwords[0].values:
            token_new.append(k)

    str_clean = ' '.join(token_new)
    return str_clean

def pra_pemrosesan(list_tweet, use_stopword=True):
    tweet_clean = []
    for tw in tqdm(list_tweet):
        normal_tweet = clean_data(tw)
        if use_stopword:
          normal_tweet = remove_stopwords_and_normalize(normal_tweet)
        tweet_clean.append(normal_tweet)
    return tweet_clean

In [ ]:
# Mengambil data
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rFLB_1QpZhKyWFGxqPo5znTEkh7dUhPf' -O dataset.csv
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1xEIQwYre1SR71uRdQuez9MDmAUIysvAG' -O stopwordsID.csv
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=19NOzXA8Voturopg_DTuiMns3s4M2IAUz' -O kamus_singkatan.csv
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VjgivEr1pxyRCuyhVifPnaReFz0yd8Us' -O colloquial-indonesian-lexicon.csv

In [ ]:
raw_tweet = selected_df['tweet']
clean_tweet = np.array(pra_pemrosesan(raw_tweet, False))
selected_df['clean_tweet'] = clean_tweet

100%|██████████| 9916/9916 [00:00<00:00, 22790.06it/s]


Pemisahan data train dan data test

In [ ]:
labeled_df = selected_df[~(selected_df['stance']=='UNLABELED')]
unlabeled_df = selected_df[selected_df['stance']=='UNLABELED']

In [ ]:
X_labeled = labeled_df['clean_tweet']
X_unlabeled = unlabeled_df['clean_tweet']
y_labeled = labeled_df['stance']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled, test_size=0.3, random_state=42)

Transformasi data tweet ke nilai numerik

In [ ]:
vectorizer_tweet = TfidfVectorizer(ngram_range=(1,4), max_features=25000)
vectorizer_tweet.fit(X_train)

X_train_labeled = vectorizer_tweet.transform(X_train).todense()
X_vect_unlabeled = vectorizer_tweet.transform(X_unlabeled).todense()
X_test_labeled = vectorizer_tweet.transform(X_test).todense()

In [ ]:
X_train_df = pd.DataFrame(data=X_train_labeled)
X_test_df = pd.DataFrame(data=X_test_labeled)
X_unlabeled_df = pd.DataFrame(data=X_vect_unlabeled)

y_train_df = pd.DataFrame(data=y_train).reset_index(drop=True)
y_test_df = pd.DataFrame(data=y_test).reset_index(drop=True)

In [ ]:
del X_train_labeled, X_test_labeled, X_unlabeled

# Data Modeling

In [ ]:
# Initiate iteration counter
iterations = 0

# Containers to hold f1_scores and # of pseudo-labels
train_f1s = []
test_f1s = []
pseudo_labels = []

# Assign value to initiate while loop
high_prob = [1] 

# Loop will run until there are no more high-probability pseudo-labels
while len(high_prob) > 0:

  over = BorderlineSMOTE()
  X_train_labeled_ovs, y_train_ovs = over.fit_resample(X_train_df, y_train_df)

  clf = MultinomialNB()
  clf.fit(X_train_labeled_ovs, y_train_ovs)
  y_hat_train = clf.predict(X_train_df)
  y_hat_test = clf.predict(X_test_df)

  # Calculate and print iteration # and f1 scores, and store f1 scores
  train_f1 = f1_score(y_train_df, y_train_df, average='macro')
  test_f1 = f1_score(y_test_df, y_hat_test, average='macro')
  print(f"Iteration {iterations}")
  print(f"Train f1: {train_f1}")
  print(f"Test f1: {test_f1}")
  train_f1s.append(train_f1)
  test_f1s.append(test_f1)

  # Generate predictions and probabilities for unlabeled data
  print(f"Now predicting labels for unlabeled data...")

  preds = clf.predict(X_unlabeled_df)
  pred_probs = clf.predict_proba(X_unlabeled_df)
  prob_0 = pred_probs[:,0]
  prob_1 = pred_probs[:,1]
  prob_2 = pred_probs[:,2]

  # Store predictions and probabilities in dataframe
  df_pred_prob = pd.DataFrame([])
  df_pred_prob['stance'] = preds
  df_pred_prob['prob_0'] = prob_0
  df_pred_prob['prob_1'] = prob_1
  df_pred_prob['prob_2'] = prob_2
  df_pred_prob.index = X_unlabeled_df.index
  
  # Separate predictions with > 90% probability
  high_prob = pd.concat([df_pred_prob.loc[df_pred_prob['prob_0'] > 0.90],
                         df_pred_prob.loc[df_pred_prob['prob_1'] > 0.90],
                         df_pred_prob.loc[df_pred_prob['prob_2'] > 0.90]],
                        axis=0)
  
  pseudo_labels.append(len(high_prob))
  print(f"{pseudo_labels[-1]} high-probability predictions added to training data.")

  # Add pseudo-labeled data to training data
  X_train_df = pd.concat([X_train_df, X_unlabeled_df.loc[high_prob.index]], axis=0, ignore_index=True)
  y_train_df = pd.concat([y_train_df, pd.DataFrame(high_prob.stance)], ignore_index=True)  
  
  # Drop pseudo-labeled instances from unlabeled data
  X_unlabeled_df = X_unlabeled_df.drop(index=high_prob.index)
  X_unlabeled_df.reset_index(drop=True)
  print(f"{len(X_unlabeled_df)} unlabeled instances remaining.\n")
  
  # Update iteration counter
  iterations += 1

Iteration 0
Train f1: 1.0
Test f1: 0.5792848315598569
Now predicting labels for unlabeled data...
4 high-probability predictions added to training data.
9487 unlabeled instances remaining.

Iteration 1
Train f1: 1.0
Test f1: 0.5809699319499957
Now predicting labels for unlabeled data...
16 high-probability predictions added to training data.
9471 unlabeled instances remaining.

Iteration 2
Train f1: 1.0
Test f1: 0.6045440821256038
Now predicting labels for unlabeled data...
352 high-probability predictions added to training data.
9119 unlabeled instances remaining.

Iteration 3
Train f1: 1.0
Test f1: 0.5283299526707235
Now predicting labels for unlabeled data...
1356 high-probability predictions added to training data.
7763 unlabeled instances remaining.

Iteration 4
Train f1: 1.0
Test f1: 0.5477673986403874
Now predicting labels for unlabeled data...
2198 high-probability predictions added to training data.
5565 unlabeled instances remaining.

Iteration 5
Train f1: 1.0
Test f1: 0.4993

In [ ]:
import pickle

# save TFIDF Vectorizer
filename = 'tfidf-vectorizer-tweet-stance.sav'
pickle.dump(vectorizer_tweet, open(path+filename, 'wb'))

# save model
filename = 'model_semisupervised-stance.sav'
pickle.dump(clf, open(path+filename, 'wb'))

In [ ]:
with open(path+'tfidf-vectorizer-tweet-stance.sav', 'rb') as vect:
  test = pickle.load(vect)

Prediksi data tanpa label

In [ ]:
pred = clf.predict(X_vect_unlabeled)
unlabeled_df['stance'] = pred

In [ ]:
unlabeled_df

,id,tweet,stance,clean_tweet
0,1528163173810860032,@dr_koko28 Solusinya vaksin lagi ?,ABSTAIN,solusinya vaksin lagi
1,1528161907710251008,Mager nya efek vaksin ini 🥲🥲🥲,KONTRA,mager nya efek vaksin ini
2,1528161749060440066,Njir udah gejala meriang gegara efek vaksin bo...,KONTRA,njir udah gejala meriang gegara efek vaksin bo...
3,1528160751445086209,Step by step.... Outputnya ya vaksin. Simpel. ...,PRO,step by step outputnya ya vaksin simpel covid ...
4,1528160134945460224,Vaksinasi Covid-19 Lansia di Kendari Sudah Mel...,PRO,vaksinasi covid lansia di kendari sudah meleb...
...,...,...,...,...
9911,1526875038070218752,"Percepat Vaksinasi, Dengan Humanis Forpimka Se...",PRO,percepat vaksinasi dengan humanis forpimka sem...
9912,1526874139381755904,VIDEO: FDA Izinkan Vaksin Booster Pfizer untuk...,PRO,video fda izinkan vaksin booster pfizer untuk ...
9913,1526873365033537536,@its_shihaa Udaah pake pfizer jumat minggu lal...,ABSTAIN,udaah pake pfizer jumat minggu lalu besoknya ...
9914,1526873332393598977,Penumpang KAJJ vaksinasi dosis kedua atau boos...,ABSTAIN,penumpang kajj vaksinasi dosis kedua atau boos...


In [ ]:
complete_label_df = pd.concat([labeled_df, unlabeled_df], axis=0, ignore_index=True)

In [ ]:
final_df = df.merge(complete_label_df[['id', 'stance']], right_on='id', left_on='id')

In [ ]:
# save final data
final_df.to_csv(path+"labeled-tweet-twint-data.csv", index=False)

In [ ]:
final_df[final_df['stance']=='ABSTAIN'].shape

(5451, 39)